This notebook is just for submission purpose.  
[Pytorch tutorial training notebook is here](https://www.kaggle.com/sanchitvj/global-wheat-detection)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import time, random, os, cv2, ast, glob, numba
from numba import jit
from tqdm.autonotebook import tqdm
from pprint import pprint
import sys

import torch, torchvision
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as T
from torchvision.models.detection.faster_rcnn import AnchorGenerator, FastRCNNPredictor, FasterRCNN
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models import mobilenet_v2, resnet101, vgg19

In [ ]:
test_dir = '../input/global-wheat-detection/test/'
test_df = pd.read_csv('../input/global-wheat-detection/sample_submission.csv')

In [ ]:
# Dataset class for evaluation.
class eval_dataset(Dataset):

    def __init__(self, dataframe, image_dir, transforms=None):
        super().__init__()

        self.image_ids = dataframe['image_id'].unique()
        self.df = dataframe
        self.image_dir = image_dir
        self.transforms = transforms

    def __getitem__(self, index: int):

        image_id = self.image_ids[index]
        records = self.df[self.df['image_id'] == image_id]

        img = cv2.imread(f'{self.image_dir}/{image_id}.jpg', cv2.IMREAD_COLOR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32)
        img /= 255.0

        if self.transforms is not None:
            img = self.transforms(img)

        return img, image_id

    def __len__(self) -> int:
        return self.image_ids.shape[0]

In [ ]:
# collate_fn is called with a list of data samples at each time.
# It is expected to collate the input samples into a batch for
# yielding from the data loader iterator.
# https://discuss.pytorch.org/t/how-to-use-collate-fn/27181
def collate_fn(batch):
    return tuple(zip(*batch))

# Custom dataloader for test data.
test_dataset = eval_dataset(test_df, test_dir, transforms=T.Compose([T.ToTensor()]))
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, num_workers=8, drop_last=False, collate_fn=collate_fn)

In [ ]:
def format_prediction_string(boxes, scores):
    pred_strings = []
    for j in zip(scores, boxes):
        pred_strings.append("{0:.4f} {1} {2} {3} {4}".format(j[0], j[1][0], j[1][1], j[1][2], j[1][3]))

    return " ".join(pred_strings)

In [ ]:
def evaluate(device, model):
    
    model.eval()
    detection_threshold = 0.5
    results = []

    for images, image_ids in test_loader:

        images = list(image.to(device) for image in images)
        outputs = model(images)

        for i, image in enumerate(images):

            boxes = outputs[i]['boxes'].data.cpu().numpy()
            scores = outputs[i]['scores'].data.cpu().numpy()

            boxes = boxes[scores >= detection_threshold].astype(np.int32)
            scores = scores[scores >= detection_threshold]
            image_id = image_ids[i]

            result = {
                'image_id': image_id,
                'PredictionString': format_prediction_string(boxes, scores)
            }

            results.append(result)
            
    return results

In [ ]:
DEVICE = torch.device('cuda')
model_path = '../input/saved-model/frcnn_best_model_epoch_8'
model = torch.load(model_path)

results = evaluate(DEVICE, model = model) 

In [ ]:
test_df = pd.DataFrame(results, columns=['image_id', 'PredictionString'])
test_df.to_csv('submission.csv', index=False)